In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import glob
import mysql.connector
from mysql.connector import Error

In [2]:
try:
    connection = mysql.connector.connect(host='attribution-pc.ccjslhbmclgf.eu-central-1.rds.amazonaws.com',
                                         database='etl_site_traffic',
                                         user='fsg',
                                         password='Attribution3.0')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("Your connected to database: ", record)
        historic = pd.read_sql('SELECT cb.Conv_ID, cb.Customer_ID, cb.Conv_Date, cb.Order_Num, cb.Revenue_excl_VAT as "Revenue", cb.Days_Since_Last FROM conversions_backend cb WHERE Account = "Paula" AND Market in  ("DE","AT")',con=connection)
except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if (connection.is_connected()):
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.23
Your connected to database:  ('etl_site_traffic',)
MySQL connection is closed


In [3]:
historic.head()

,Conv_ID,Customer_ID,Conv_Date,Order_Num,Revenue,Days_Since_Last
0,500000036,4a32081c69f5f1f0e38b05a6db711b8c,2009-05-13,0,9.00,-1
1,500000038,cf74c231fca1c57e8e695f18864111b0,2009-05-14,0,39.38,-1
2,500000039,c77baabec952c8675252b7041ebdea41,2009-05-15,0,61.25,-1
3,500000049,bcc3cd3fddbea92b5faa39f29aab8b69,2009-05-17,0,25.79,-1
4,500000066,e7b5709b18b569fff502aff25d8fe9f2,2009-05-24,0,35.21,-1


In [4]:
historic = historic[historic['Revenue']>0.0]

In [5]:
historic['Conv_Date'] = pd.to_datetime(historic['Conv_Date'], format="%Y-%m-%d")

In [6]:
new_c = (historic['Order_Num']==0)
ret_c = (historic['Order_Num']>0)

In [7]:
historic['Customer_Type'] = ""

In [8]:
historic.loc[new_c,['Customer_Type']] = "New"
historic.loc[ret_c,['Customer_Type']] = "Returning"

In [10]:
historic_apr19 = historic[historic['Conv_Date']<= '2019-04-14']
historic_oct19 = historic[historic['Conv_Date']<= '2019-10-13']
historic_apr20 = historic[historic['Conv_Date']<= '2020-04-12']
historic_oct20 = historic[historic['Conv_Date']<= '2020-10-12']
historic_apr21 = historic[historic['Conv_Date']<= '2021-04-12']
historic_oct21 = historic[historic['Conv_Date']<= '2021-10-11']

In [11]:
time_diff_apr19 = historic_apr19.groupby('Customer_ID')['Conv_Date'].agg(['max','min']).reset_index()
time_diff_oct19 = historic_oct19.groupby('Customer_ID')['Conv_Date'].agg(['max','min']).reset_index()
time_diff_apr20 = historic_apr20.groupby('Customer_ID')['Conv_Date'].agg(['max','min']).reset_index()
time_diff_oct20 = historic_oct20.groupby('Customer_ID')['Conv_Date'].agg(['max','min']).reset_index()
time_diff_apr21 = historic_apr21.groupby('Customer_ID')['Conv_Date'].agg(['max','min']).reset_index()
time_diff_oct21 = historic_oct21.groupby('Customer_ID')['Conv_Date'].agg(['max','min']).reset_index()

In [12]:
time_diff_apr19['Tenure'] = (time_diff_apr19['max']-time_diff_apr19['min']).dt.days
time_diff_oct19['Tenure'] = (time_diff_oct19['max']-time_diff_oct19['min']).dt.days
time_diff_apr20['Tenure'] = (time_diff_apr20['max']-time_diff_apr20['min']).dt.days
time_diff_oct20['Tenure'] = (time_diff_oct20['max']-time_diff_oct20['min']).dt.days
time_diff_apr21['Tenure'] = (time_diff_apr21['max']-time_diff_apr21['min']).dt.days
time_diff_oct21['Tenure'] = (time_diff_oct21['max']-time_diff_oct21['min']).dt.days

In [13]:
historic_apr19 = historic_apr19.merge(time_diff_apr19, on='Customer_ID',how='inner')
historic_oct19 = historic_oct19.merge(time_diff_oct19, on='Customer_ID',how='inner')
historic_apr20 = historic_apr20.merge(time_diff_apr20, on='Customer_ID',how='inner')
historic_oct20 = historic_oct20.merge(time_diff_oct20, on='Customer_ID',how='inner')
historic_apr21 = historic_apr21.merge(time_diff_apr21, on='Customer_ID',how='inner')
historic_oct21 = historic_oct21.merge(time_diff_oct21, on='Customer_ID',how='inner')

In [14]:
historic_apr19['Tenure_Bucket'] = ""
historic_oct19['Tenure_Bucket'] = ""
historic_apr20['Tenure_Bucket'] = ""
historic_oct20['Tenure_Bucket'] = ""
historic_apr21['Tenure_Bucket'] = ""
historic_oct21['Tenure_Bucket'] = ""

In [15]:
def tenure_bucket(col):
    if (col <= 30) :
        return 'Less than 1 Month'
    elif ((col > 30) & (col <= 90)):
        return 'Between 1 and 3 Months'
    elif ((col > 90) & (col <= 180)):
        return 'Between 3 and 6 Months'    
    elif ((col > 180) & (col <= 365)):
        return 'Between 6 and 12 Months'
    else:
        return 'Over 12 Months'

In [16]:
historic_apr19.loc[:,['Tenure_Bucket']] = historic_apr19['Tenure'].apply(lambda x: tenure_bucket(x))
historic_oct19.loc[:,['Tenure_Bucket']] = historic_oct19['Tenure'].apply(lambda x: tenure_bucket(x))
historic_apr20.loc[:,['Tenure_Bucket']] = historic_apr20['Tenure'].apply(lambda x: tenure_bucket(x))
historic_oct20.loc[:,['Tenure_Bucket']] = historic_oct20['Tenure'].apply(lambda x: tenure_bucket(x))
historic_apr21.loc[:,['Tenure_Bucket']] = historic_apr21['Tenure'].apply(lambda x: tenure_bucket(x))
historic_oct21.loc[:,['Tenure_Bucket']] = historic_oct21['Tenure'].apply(lambda x: tenure_bucket(x))

In [17]:
gsw_apr19_all = historic_apr19[(historic_apr19['Conv_Date']>= '2019-04-06') & (historic_apr19['Conv_Date']<= '2019-04-14')]
gsw_oct19_all = historic_oct19[(historic_oct19['Conv_Date']>= '2019-10-05') & (historic_oct19['Conv_Date']<= '2019-10-13')]
gsw_apr20_all = historic_apr20[(historic_apr20['Conv_Date']>= '2020-04-03') & (historic_apr20['Conv_Date']<= '2020-04-12')]
gsw_oct20_all = historic_oct20[(historic_oct20['Conv_Date']>= '2020-10-02') & (historic_oct20['Conv_Date']<= '2020-10-11')]
gsw_apr21_all = historic_apr21[(historic_apr21['Conv_Date']>= '2021-04-03') & (historic_apr21['Conv_Date']<= '2021-04-12')]
gsw_oct21_all = historic_oct21[(historic_oct21['Conv_Date']>= '2021-10-02') & (historic_oct21['Conv_Date']<= '2021-10-10')]

In [18]:
gsw_apr19_ret = gsw_apr19_all[gsw_apr19_all['Customer_Type']=='Returning']
gsw_oct19_ret = gsw_oct19_all[gsw_oct19_all['Customer_Type']=='Returning']
gsw_apr20_ret = gsw_apr20_all[gsw_apr20_all['Customer_Type']=='Returning']
gsw_oct20_ret = gsw_oct20_all[gsw_oct20_all['Customer_Type']=='Returning']
gsw_apr21_ret = gsw_apr21_all[gsw_apr21_all['Customer_Type']=='Returning']
gsw_oct21_ret = gsw_oct21_all[gsw_oct21_all['Customer_Type']=='Returning']

In [19]:
gsw_apr19_ret.groupby('Tenure_Bucket').agg({'Conv_ID':pd.Series.nunique}).reset_index().to_clipboard(index=False)

In [20]:
new_cust = historic[historic['Customer_Type']=='New']

In [21]:
gsw_apr19 = new_cust[(new_cust['Conv_Date']>= '2019-04-06') & (new_cust['Conv_Date']<= '2019-04-14')]
gsw_oct19 = new_cust[(new_cust['Conv_Date']>= '2019-10-05') & (new_cust['Conv_Date']<= '2019-10-13')]
gsw_apr20 = new_cust[(new_cust['Conv_Date']>= '2020-04-03') & (new_cust['Conv_Date']<= '2020-04-12')]
gsw_oct20 = new_cust[(new_cust['Conv_Date']>= '2020-10-02') & (new_cust['Conv_Date']<= '2020-10-11')]
gsw_apr21 = new_cust[(new_cust['Conv_Date']>= '2021-04-03') & (new_cust['Conv_Date']<= '2021-04-12')]
gsw_oct21 = new_cust[(new_cust['Conv_Date']>= '2021-10-02') & (new_cust['Conv_Date']<= '2021-10-10')]

In [22]:
gsw_apr19['AOV_Bucket'] = ""
gsw_oct19['AOV_Bucket'] = ""
gsw_apr20['AOV_Bucket'] = ""
gsw_oct20['AOV_Bucket'] = ""
gsw_apr21['AOV_Bucket'] = ""
gsw_oct21['AOV_Bucket'] = ""

<ipython-input-22-6522d3483d24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gsw_apr19['AOV_Bucket'] = ""
<ipython-input-22-6522d3483d24>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gsw_oct19['AOV_Bucket'] = ""
<ipython-input-22-6522d3483d24>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [23]:
rfm_apr19 = gsw_apr19_ret.groupby(['Customer_ID']).agg({'Order_Num':pd.Series.max,'Days_Since_Last':pd.Series.min,'Revenue':pd.Series.mean}).reset_index()
rfm_oct19 = gsw_oct19_ret.groupby(['Customer_ID']).agg({'Order_Num':pd.Series.max,'Days_Since_Last':pd.Series.min,'Revenue':pd.Series.mean}).reset_index()
rfm_apr20 = gsw_apr20_ret.groupby(['Customer_ID']).agg({'Order_Num':pd.Series.max,'Days_Since_Last':pd.Series.min,'Revenue':pd.Series.mean}).reset_index()
rfm_oct20 = gsw_oct20_ret.groupby(['Customer_ID']).agg({'Order_Num':pd.Series.max,'Days_Since_Last':pd.Series.min,'Revenue':pd.Series.mean}).reset_index()
rfm_apr21 = gsw_apr21_ret.groupby(['Customer_ID']).agg({'Order_Num':pd.Series.max,'Days_Since_Last':pd.Series.min,'Revenue':pd.Series.mean}).reset_index()
rfm_oct21 = gsw_oct21_ret.groupby(['Customer_ID']).agg({'Order_Num':pd.Series.max,'Days_Since_Last':pd.Series.min,'Revenue':pd.Series.mean}).reset_index()

In [24]:
# Purchase Frequency
purchase_frequency_apr19=sum(rfm_apr19['Order_Num'])/rfm_apr19.shape[0]
purchase_frequency_oct19=sum(rfm_oct19['Order_Num'])/rfm_oct19.shape[0]
purchase_frequency_apr20=sum(rfm_apr20['Order_Num'])/rfm_apr20.shape[0]
purchase_frequency_oct20=sum(rfm_oct20['Order_Num'])/rfm_oct20.shape[0]
purchase_frequency_apr21=sum(rfm_apr21['Order_Num'])/rfm_apr21.shape[0]
purchase_frequency_oct21=sum(rfm_oct21['Order_Num'])/rfm_oct21.shape[0]

In [25]:
# Repeat Rate
repeat_rate_apr19=rfm_apr19[rfm_apr19.Order_Num > 1].shape[0]/rfm_apr19.shape[0]
repeat_rate_oct19=rfm_oct19[rfm_oct19.Order_Num > 1].shape[0]/rfm_oct19.shape[0]
repeat_rate_apr20=rfm_apr20[rfm_apr20.Order_Num > 1].shape[0]/rfm_apr20.shape[0]
repeat_rate_oct20=rfm_oct20[rfm_oct20.Order_Num > 1].shape[0]/rfm_oct20.shape[0]
repeat_rate_apr21=rfm_apr21[rfm_apr21.Order_Num > 1].shape[0]/rfm_apr21.shape[0]
repeat_rate_oct21=rfm_oct21[rfm_oct21.Order_Num > 1].shape[0]/rfm_oct21.shape[0]

In [26]:
# Churn Rate
churn_rate_apr19=1-repeat_rate_apr19
churn_rate_oct19=1-repeat_rate_oct19
churn_rate_apr20=1-repeat_rate_apr20
churn_rate_oct20=1-repeat_rate_oct20
churn_rate_apr21=1-repeat_rate_apr21
churn_rate_oct21=1-repeat_rate_oct21

In [27]:
# Customer Value
rfm_apr19['CLV']=(rfm_apr19['Revenue']*purchase_frequency_apr19)/churn_rate_apr19
rfm_oct19['CLV']=(rfm_oct19['Revenue']*purchase_frequency_oct19)/churn_rate_oct19
rfm_apr20['CLV']=(rfm_apr20['Revenue']*purchase_frequency_apr20)/churn_rate_apr20
rfm_oct20['CLV']=(rfm_oct20['Revenue']*purchase_frequency_oct20)/churn_rate_oct20
rfm_apr21['CLV']=(rfm_apr21['Revenue']*purchase_frequency_apr21)/churn_rate_apr21
rfm_oct21['CLV']=(rfm_oct21['Revenue']*purchase_frequency_oct21)/churn_rate_oct21

In [28]:
# Define scoring functions for segmentation

def recency_core(x,p,df):
    if x <= df[p][0.20]:
        return 5
    elif x <= df[p][0.4]:
        return 4
    elif x <= df[p][0.6]: 
        return 3
    elif x <= df[p][0.8]: 
        return 2
    else:
        return 1
    
def freq_monetary_score(x,p,df):
    if x <= df[p][0.20]:
        return 1
    elif x <= df[p][0.40]:
        return 2
    elif x <= df[p][0.60]: 
        return 3
    elif x <= df[p][0.80]: 
        return 4
    else:
        return 5

In [29]:
# Split metrics into segments is by using quartiles
quantiles_apr19 = rfm_apr19.quantile(q=[0.2,0.4,0.6,0.8])
quantiles_apr19 = quantiles_apr19.to_dict()

quantiles_oct19 = rfm_oct19.quantile(q=[0.2,0.4,0.6,0.8])
quantiles_oct19 = quantiles_oct19.to_dict()

quantiles_apr20 = rfm_apr20.quantile(q=[0.2,0.4,0.6,0.8])
quantiles_apr20 = quantiles_apr20.to_dict()

quantiles_oct20 = rfm_oct20.quantile(q=[0.2,0.4,0.6,0.8])
quantiles_oct20 = quantiles_oct20.to_dict()

quantiles_apr21 = rfm_apr21.quantile(q=[0.2,0.4,0.6,0.8])
quantiles_apr21 = quantiles_apr21.to_dict()

quantiles_oct21 = rfm_oct21.quantile(q=[0.2,0.4,0.6,0.8])
quantiles_oct21 = quantiles_oct21.to_dict()

In [30]:
quantiles_apr19

{'Order_Num': {0.2: 2.0, 0.4: 5.0, 0.6: 8.0, 0.8: 15.0},
 'Days_Since_Last': {0.2: 31.0, 0.4: 61.0, 0.6: 140.0, 0.8: 198.0},
 'Revenue': {0.2: 31.6, 0.4: 50.42, 0.6: 71.6, 0.8: 97.48},
 'CLV': {0.2: 2027.298934911242,
  0.4: 3234.696591715976,
  0.6: 4593.500118343193,
  0.8: 6253.832284023667}}

In [31]:
purchase_frequency_oct21

7.69731037427067

In [32]:
gsw_oct21_ret.head()

,Conv_ID,Customer_ID,Conv_Date,Order_Num,Revenue,Days_Since_Last,Customer_Type,max,min,Tenure,Tenure_Bucket
328,EU03600904,9b731e44cc6d931911b69a72fa376815,2021-10-03,4,64.55,58,Returning,2021-10-03,2010-10-12,4009,Over 12 Months
388,EU03611498,7aadfc0b957a0a42df7a8ba100ca0317,2021-10-06,10,131.11,133,Returning,2021-10-06,2010-12-07,3956,Over 12 Months
869,EU03618991,f3d3f5f04173c31e962af86caf0bc0b3,2021-10-08,35,135.54,12,Returning,2021-10-08,2012-02-04,3534,Over 12 Months
1255,EU03617323,841d5a82d3df20fd816f651f302b844a,2021-10-08,20,26.22,37,Returning,2021-10-08,2012-09-23,3302,Over 12 Months
1654,EU03597209,953bcccc4b24bb046cc64e82c403ef37,2021-10-02,39,159.36,84,Returning,2021-10-02,2012-11-30,3228,Over 12 Months
